In [1]:
from langchain.agents import create_agent
from langchain.tools import tool
import os

In [ ]:
# 验证是否有Open-Ai key
print("API KEY FOUND:", bool(os.getenv("OPENAI_API_KEY")))

API KEY FOUND: True


In [ ]:

# decoration
@tool
# 大模型会看到类似这样的描述
# {
#   "name": "get_weather",
#   "description": "Get weather for a given city.",
#   "parameters": {
#     "city": "string"
#   }
# }

def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

agent = create_agent(
    "openai:gpt-4.1-mini",   # 也可以用 "openai:gpt-4o" / "openai:gpt-5"
    tools=[get_weather],
    system_prompt="You are a helpful assistant",
)

# # Run the agent
result = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]}
)
print(result)


{'messages': [HumanMessage(content='what is the weather in sf', additional_kwargs={}, response_metadata={}, id='1c1ba17a-5ef0-4f03-a59d-5979c3c85b52'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 56, 'total_tokens': 71, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_d0c93c37b1', 'id': 'chatcmpl-CmwoR5zWdhnbc96rGCe4NEHIgcGvi', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b20cc-da79-7ff3-9391-46b1bb0d3824-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'San Francisco'}, 'id': 'call_vvfhE8cWRHgeRulz1ciazDJx', 'type': 'tool_call'}], usage_metadata={'input_tokens': 56, 'output_tokens': 15, 'total

### agent.invoke中发生了什么

#### 1) invoke() 是什么？

agent.invoke(...) 的意思是：

“用一次输入，驱动这个 agent 跑完一整个回合（turn）。”

这个回合通常包含：

把输入整理成模型可用的格式

让模型思考：要不要调用工具

如果要，执行工具函数

把工具结果再喂回模型

得到最终回答（自然语言）

所以 invoke 不是“只问模型一句话”，而是可能包含 多步推理 + 多次模型调用 + 工具执行 的完整流程

#### 2) 为什么参数要包成 {"messages": [...]}？

你传进去的是一个 dict：

```python
{
  "messages": [
    {"role": "user", "content": "what is the weather in sf"}
  ]
}
```

这是“聊天式模型”（Chat Model）的标准输入形态：一段对话历史。

messages：对话列表

每条消息有：

role：消息角色（system / user / assistant / tool）

content：文本内容

你这里只给了一条 user 消息，表示“新对话的第一句”。

关键点：
Agent 需要对话历史，因为它要把工具调用的结果（tool messages）也插回到对话里，让模型继续“接着说”。

#### 3) role: "user" / content: ... 分别代表什么？
role: "user"

表示这句话是“用户说的”。模型会用不同方式对待不同角色：

system：最高优先级的规则/身份设定（你在 system_prompt 里写的内容最终会变成 system message）

user：用户需求

assistant：模型先前说过的话（用于保持上下文一致）

tool：工具返回的结构化结果（让模型基于真实输出继续）

content: "what is the weather in sf"

这是真正的问题文本。
“sf” 可能会被模型理解成 San Francisco，也可能需要进一步确认；但在你的 toy tool 里，它通常会直接把 "sf" 作为参数传给工具。

#### 4) Agent 收到这份输入后，内部典型流程是什么？

下面按“典型 tool-calling agent”的逻辑走（不同版本实现细节略有差异，但核心一样）：

##### Step A：拼装完整 prompt / 对话

Agent 会把你创建 agent 时的系统提示加入进去，例如：

system: "You are a helpful assistant"

user: "what is the weather in sf"

同时，Agent 还会把“可用工具列表”告诉模型（这是 tool-calling 的关键），概念上像这样：

Tools available: get_weather(city: str) - Get weather for a given city.

（具体注入方式由 LangChain 版本决定。）

##### Step B：模型第一次输出：决定“要不要调用工具”

模型看到“weather in sf”，一般会判断：

这是查天气任务，应该调用工具 get_weather

于是它不会直接给你最终自然语言答案，而是先输出一个 工具调用指令（概念化）：

{
  "tool_call": {
    "name": "get_weather",
    "arguments": {"city": "sf"}
  }
}


注意：
模型在这一步并没有执行 Python 函数。
它只是“生成一个结构化意图”：要调用哪个工具，参数是什么。

##### Step C：LangChain 执行你写的 Python 工具函数

LangChain 看到模型请求调用：

tool name: get_weather

args: city="sf"

就真的在 Python 里运行：

get_weather("sf")


得到返回值：

"It's always sunny in sf!"

##### Step D：把工具输出作为 tool 消息追加回对话

这是 Agent 能“用工具结果继续说”的关键一步。

对话历史会被扩展成概念上这样：

system: You are a helpful assistant

user: what is the weather in sf

tool: It's always sunny in sf!

（真实结构可能含 tool_call_id 等字段。）

Step E：模型第二次输出：生成最终回答

模型现在拿到了工具的真实输出，于是会生成最终自然语言回答，例如：

“It’s always sunny in sf!”

然后 invoke() 把这个最终结果返回给你。

5) agent.invoke(...) 返回的到底是什么？

取决于你用的 Agent API/版本：

有的返回一个对象/字典（包含中间步骤、messages、output）

有的返回最终的 AIMessage

如果你用了 AgentExecutor(verbose=True)，还会在控制台打印中间推理/工具调用信息

所以你常见的拿法会是：
```python
result = agent.invoke(...)
print(result)
```

或者（某些 executor 风格）：
```python
print(result["output"])
```
